In [1]:
# installs notebook importer and imports it
# is needed to import functions from other notebooks
def install_pip_pkg(required : set):
    import sys
    import subprocess
    import pkg_resources

    installed = {pkg.key for pkg in pkg_resources.working_set}
    missing = required - installed

    if missing:
        python = sys.executable
        subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

#%pip install nbimporter
install_pip_pkg({'nbimporter'})
import nbimporter

In [3]:
import sys, os
sys.path.append(os.path.join(os.path.dirname(''), '..'))
#sys.path.append(os.path.join(os.path.dirname(''), '../..'))
from proj3_gans_scooters.src.proj3_airport_scraping import icao_airport_codes
from proj3_gans_scooters.src.proj3_city_scraping import scrap_wiki_cities
from proj3_gans_scooters.src.proj3_weather_scraping import scrape_weather
from proj3_gans_scooters.src.proj3_utils import load_keys

In [4]:
relative_path_to_file = '.env'
api_key_dict = load_keys(relative_path_to_file)

NameError: name 'ConfigParser' is not defined

In [6]:
cities_df = scrap_wiki_cities()

In [7]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   city                94 non-null     object
 1   member_state        94 non-null     object
 2   officialpopulation  94 non-null     object
 3   date                94 non-null     object
dtypes: object(4)
memory usage: 3.1+ KB


In [8]:
weather_df = scrape_weather(list(cities_df.city), api_key_dict['openweather_key'])

Das System kann den angegebenen Pfad nicht finden.


In [9]:
weather_df

,city,date,temp_celcius,temp_feels_like_celcius,humidity_percent,weather_description,clouds_percent,wind_speed_meter_sec,wind_direction_degree,wind_gust_meter_sec,pop_percent,rain_3h_mm,pod
0,Berlin,2022-06-08 12:00:00,24.14,23.84,47,light rain,40,3.97,215,6.18,0.55,0.42,d
1,Berlin,2022-06-08 15:00:00,23.13,22.78,49,broken clouds,77,1.73,228,4.74,0.10,0.00,d
2,Berlin,2022-06-08 18:00:00,21.77,21.39,53,overcast clouds,98,2.07,166,4.11,0.03,0.00,d
3,Berlin,2022-06-08 21:00:00,18.86,18.52,66,light rain,98,1.90,141,2.97,0.36,0.13,n
4,Berlin,2022-06-09 00:00:00,15.48,15.38,88,light rain,99,2.85,287,5.86,0.41,0.77,n
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3755,Vila Nova de Gaia,2022-06-12 21:00:00,21.41,21.59,76,broken clouds,60,0.94,302,1.73,0.00,0.00,n
3756,Vila Nova de Gaia,2022-06-13 00:00:00,20.62,20.77,78,broken clouds,70,1.09,179,1.43,0.00,0.00,n
3757,Vila Nova de Gaia,2022-06-13 03:00:00,20.35,20.45,77,clear sky,1,1.13,132,1.40,0.00,0.00,n
3758,Vila Nova de Gaia,2022-06-13 06:00:00,20.55,20.70,78,clear sky,1,0.79,148,1.31,0.00,0.00,d


In [10]:

icao_airports_df = icao_airport_codes(latitudes, longitudes, querystring, api_key_dict['aerodatabox_key'], headers)

NameError: name 'latitudes' is not defined